In [1]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

c:\Users\TVT\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load pre-trained Pegasus model and tokenizer
model_name = "google/pegasus-cnn_dailymail"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Load dataset for fine-tuning (e.g., CNN/DailyMail dataset)
dataset = load_dataset("cnn_dailymail", "3.0.0")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Tokenize dataset for training
def tokenize_function(example):
    source_text = example["article"]
    target_text = example["highlights"]
    source_tokenized = tokenizer(source_text, truncation=True, padding="max_length", max_length=1024, return_tensors="pt")
    target_tokenized = tokenizer(target_text, truncation=True, padding="max_length", max_length=150, return_tensors="pt")
    return {
        "input_ids": source_tokenized.input_ids,
        "attention_mask": source_tokenized.attention_mask,
        "labels": target_tokenized.input_ids,
    }

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [4]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    logging_dir="./logs",
    logging_steps=1000,
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

c:\Users\TVT\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [5]:
# Fine-tune the model
trainer.train()

  0%|          | 2/215337 [04:56<9054:40:22, 151.38s/it]

KeyboardInterrupt: 

In [ ]:
# Save the fine-tuned model
model.save_pretrained("./fine_tuned_model")

In [ ]:
# Example of generating summaries using the fine-tuned model
input_text = "Your input text goes here."
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
generated_summary_ids = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
generated_summary = tokenizer.decode(generated_summary_ids[0], skip_special_tokens=True)
print("Generated Summary:")
print(generated_summary)